### Описание проекта
* В нашем распоряжении данные сервиса Яндекс Недвижимость — архив объявлений за несколько лет о продаже квартир в Санкт-Петербурге и соседних населённых пунктах.
* Наша задача — выполнить предобработку данных и изучить их, чтобы найти интересные особенности и зависимости, которые существуют на рынке недвижимости.
* О каждой квартире в базе содержится два типа данных: добавленные пользователем и картографические. Например, к первому типу относятся площадь квартиры, её этаж и количество балконов, ко второму — расстояния до центра города, аэропорта и ближайшего парка.

### Задачи проекта
* Изучить параметры объектов.
* Изучить, как быстро продавались квартиры.
* Определить какие факторы больше всего влияют на общую стоимость объекта.
* Посчитайть среднюю цену одного квадратного метра в 10 населённых пунктах с наибольшим числом объявлений. Выделите населённые пункты с самой высокой и низкой стоимостью квадратного метра.
* Выделить квартиры в зависимости от населенного пункта и вычислить среднюю цену каждого километра до центра города. Описать, как стоимость объектов зависит от расстояния до центра города.

**1 Открываем файл с данными и изучаем общую информацию.**

In [ ]:
# загружаем необходимые библиотеки

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# загрузим файл и выведем первые 10 строк

df = pd.read_csv('/kaggle/input/real-estate-spb/real_estate_data.csv', sep='\t')
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
df.head(10)

In [ ]:
df.info()

In [ ]:
# считаем общее количество строк и столбцов

df.shape 

**В дата сете указаны объкты недвижимости со следующими параметрами, всего колонок 22:**
- airports_nearest — расстояние до ближайшего аэропорта в метрах (м)
- balcony — число балконов
- ceiling_height — высота потолков (м)
- cityCenters_nearest — расстояние до центра города (м)
- days_exposition — сколько дней было размещено объявление (от публикации до снятия)
- first_day_exposition — дата публикации
- floor — этаж
- floors_total — всего этажей в доме
- is_apartment — апартаменты (булев тип)
- kitchen_area — площадь кухни в квадратных метрах (м²)
- last_price — цена на момент снятия с публикации
- living_area — жилая площадь в квадратных метрах (м²)
- locality_name — название населённого пункта
- open_plan — свободная планировка (булев тип)
- parks_around3000 — число парков в радиусе 3 км
- parks_nearest — расстояние до ближайшего парка (м)
- ponds_around3000 — число водоёмов в радиусе 3 км
- ponds_nearest — расстояние до ближайшего водоёма (м)
- rooms — число комнат
- studio — квартира-студия (булев тип)
- total_area — общая площадь квартиры в квадратных метрах (м²)
- total_images — число фотографий квартиры в объявлении

**Всего записей об обьектах недвижимости:** *23699*

Приведем названия колонок к единому стандарту, перепишем названия в "змеином регистре"

In [ ]:
df = df.rename(columns={'cityCenters_nearest': 'city_centers_nearest', 'parks_around3000': 'parks_around_3000', 'ponds_around3000': 'ponds_around_3000'})

In [ ]:
# проверям название колонок

df.columns

Воспользуемся методом описательной статистики.

In [ ]:
df.describe()

- Обратим внимание на данные в столбце ceiling_height, максимальное значение высоты потолка равно 100 м, это не сильно влияет на соотношение медианы и среднего арифметического, в дальнейшем определим характер данной аномалии.
- В колонке с информацие о сроке размещения обьявления сильный перекос по показателю mean (6 месяцев), что означает наличие множественных обьявлений длительным сроком размещения или обусловлены высокой ценой, либо которые не актуализируются. Медианное значение составляет 3 месяца, в дальнейшем рассмотрим наличие зависимости между сроком обьявления и стоимости обьекта недвижимости.

**Построим гистограмму для всех числовых столбцов и посмотрим как распределены данные.**

In [ ]:
df[['last_price', 'total_area', 'living_area', 'kitchen_area', 'rooms', 'balcony', 'ceiling_height', 'floor', 'floors_total',
    'days_exposition']].hist(bins=100, figsize=(20,12), grid=False);

Предоставленные данные нормально распределены, нет сильно выделяющихся баров в гистограммах. Далее рассмотрим данные более детально, а пока подготовим датасет для дальнейшей работы.

**2 Предобработка данных.**

2.1 Обработаем пропуски в таблице данных о недвижимости.

In [ ]:
# посчитаем количество дубликатов

df.duplicated().sum()

Дубликаты отсутствуют.

In [ ]:
# посчитаем количество пропусков в данных для каждого столбца

df.isna().sum().sort_values(ascending=False)

In [ ]:
# представим данные в виде долей.

df.isna().mean().sort_values(ascending=False)

Из таблицы видно, что наибольшее количество пропусков в столбце is_apartment. Доля апартаментов в общем пуле обьектов недвижимости составляет около 0,02%, что не соответсвтует статистике, ведь на конец 2019 года доля апартаментов в Санкт-Петербурге составляла <a href="https://www.novostroy-spb.ru/novosti/apartamenty_zanyali_desyatuyu_chast#:~:text=К%20концу%202019%20года%20доля,скорее%20всего%2C%20он%20будет%20увеличиваться" title='Портал'>10%</a>. В дальнейшем не удастся выявиться взаимосвязь типа неджимимости и стоимости, но и такой задачи в исследовании нет, столбец можно удалить. В дальнейшем поэтапно разберем каждую категорию пропусков и определим методы заменты пропусков.

In [ ]:
# удаляем лишние столбцы

df = df.drop(columns=['total_images', 'is_apartment'])

Обработаем пропущенные значения высоты потолков. Сначала определим медианное значение и средне значение и дальше сопоставим данные со специальными стандартами устанавливающие минимальную высоту потолков в 2,5 метра.

In [ ]:
df['ceiling_height'].mean()

In [ ]:
df['ceiling_height'].median()

Ближе значению соответсвующему специальному стандарту получилось значение медианы, заменим ей пропущенные значения.

In [ ]:
df['ceiling_height'] = df['ceiling_height'].fillna(df['ceiling_height'].median())

Применим метод замены пропущенных значений нулями к столбцам balcony и floors_total, поскольку пропуски носят не случайный характер т.к. при отсутствии балкона поле скорее всего не заполнялось, а доля пропусков в количестве этаже настолько мала, замена пропусков не повлияет на выборку.

In [ ]:
df['balcony'] = df['balcony'].fillna(0)

In [ ]:
df['floors_total'] = df['floors_total'].fillna(0)

In [ ]:
# пропуски в названиях населенных пунков заполним словом Неизвестно.

df['locality_name'] = df['locality_name'].fillna('Неизвестно')

Пропуски в размерах жилой площади и кухни заменим медианой по количеству комнат.

In [ ]:
for i in df['rooms'].unique():
    df.loc[(df['rooms'] == i) & (df['living_area'].isna()), 'living_area'] = \
    df.loc[(df['rooms'] == i), 'living_area'].mean()

In [ ]:
for i in df['rooms'].unique():
    df.loc[(df['rooms'] == i) & (df['kitchen_area'].isna()), 'kitchen_area'] = \
    df.loc[(df['rooms'] == i), 'kitchen_area'].mean()

Доля пропусков следующих столбцов существенная, удалив строки мы лишимся важной статистической информации. Поэтому заменим пропуски на нули.

In [ ]:
df.fillna(value={'parks_around_3000': 0, 'ponds_around_3000': 0, 'city_centers_nearest': 0, 'airports_nearest': 0,
    'ponds_nearest': 0, 'parks_nearest': 0, 'days_exposition': 0}, inplace=True)

In [ ]:
df.isna().sum().sort_values(ascending=False)

2.2 Изменим тип данных, где это необходимо.

In [ ]:
df.info()

In [ ]:
# поменяем значения first_day_exposition на формат даты

df['first_day_exposition'] = pd.to_datetime(df['first_day_exposition'], format='%Y-%m-%dT%H:%M:%S')

# приведет остальные значения к целочисленным значениям, где не требуются значения после запятой

column_list = ['floors_total', 'balcony','last_price', 'airports_nearest', 'city_centers_nearest',
    'parks_around_3000', 'parks_nearest', 'ponds_around_3000', 'ponds_nearest', 'days_exposition']
for column in column_list:
    df[column] = df[df[column].notna()][column].astype('int64')

In [ ]:
df.info()

2.3 Изучим столбец с названиями населенных пунктов и устраним неявные дубликаты.

In [ ]:
# получим список всех уникальных значений

df['locality_name'].sort_values().unique()

In [ ]:
df = df.apply(lambda x: x.replace({'деревня Кудрово': 'Кудрово', 'поселок Мурино': 'Мурино','посёлок': 'поселок', 'городской поселок': 'поселок', 'городской посёлок': 'поселок',
    'поселок городского типа': 'поселок', 'посёлок городского типа': 'поселок', 'коттеджный посёлок': 'поселок',
    'коттеджный поселок': 'поселок', 'посёлок при железнодорожной станции': 'поселок станции', 'посёлок станции': 'поселок станции',
    'садоводческое некоммерческое товарищество': 'садовое товарищество'}, regex=True))

Привели названия населенных пунктов к одному виду, заменив названия "посёлок" на "поселок" и т.д.

пригодится более универсальный способ для замены:

for i in data['locality_name'].unique().tolist():
    data.loc[data['locality_name'] == i, 'locality_name'] = i.split()[-1] # оставляем только наименования населенного пункта

In [ ]:
# колчичество населенных пунктов

len(df['locality_name'].unique())

2.4 Найдем редкие и выбивающиеся значения.

In [ ]:
# из описательной статистики мы увидели аномальные значения в столбце ceiling_height, рассмотрим далее

df.sort_values(by='ceiling_height', ascending=False).head(10)

Исправим аномальные значения высоты потолка, к примеру высота потолка "27" метров есть высота "2,7" метра. Разделим все значения высоты потолка превышаюшие 20 метров на 10.

In [ ]:
df.loc[(df['ceiling_height'] >= 20) & (~pd.isna(df['ceiling_height'])),'ceiling_height']  = df['ceiling_height'] / 10

Проанализируем важные для исследования параметры, такие как стоимость квартиры, количество комнат и общую площать. В данных имеются аномально высокие значения это было видно на гистограммах и зачениях описательной статистики, достаточно взглянуть на максимальные значения диапазона и разницу между медианой и средне арифметическим, это говорит о том что присутствуют выбросы. Воспользуемся методом построения диаграмм размаха.

In [ ]:
# диаграмма размаха для цены объекта

plt.ylim(0, 20000000)
df.boxplot(['last_price'])

In [ ]:
# диаграмма размаха для количества комнат

plt.ylim(0, 20)
df.boxplot(['rooms'])

In [ ]:
# диаграмма размаха для общей площади

plt.ylim(0, 200)
df.boxplot(['total_area'])

* На диаграмме размаха видно большое количество выбросов за пределами нормальных значений, начиная с цены 15 млн. будем считать это верхней границей. Медианное значение составляет около 5 млн.
* На диаграмме количества комнат предел нормальных значений - 6 комнат, свыше идут выбросы вплоть до 19 комнат, эти данные также исключим из выборки.
* Общая площать составляет от 40 до 60 м, с медианой 55. Разброс значений от 10 до 120 метро за объект недвижимости.

In [ ]:
# обновим данные датасета в соотвествии с новыми параметрами

good_df = df[(df['last_price'] >=1000000) & (df['last_price'] <=15000000) & \
            (df['rooms'] >=1) & (df['rooms'] <= 6)  & \
            (df['total_area'] >=10) & (df['total_area'] <=120)]

In [ ]:
good_df.shape[0] / df.shape[0]

Отбросили порядка 8% данных, что является допустимым.

**3 Добавим в таблицу новые столбцы**

Дополним дата сет новыми столбцами, добавим количественные и категориальные значения для выявления взаимосвязей.

In [ ]:
# столбец с информацией о цене за квадратный метр

good_df['price_qm'] = np.round(good_df['last_price']/good_df['total_area'])

In [ ]:
# столбцы о годах, месяцах и днях. Например определить, в каком месяце больше всего продаж

good_df['day'] = good_df['first_day_exposition'].dt.weekday
good_df['month'] = good_df['first_day_exposition'].dt.month
good_df['year'] = good_df['first_day_exposition'].dt.year

# создадим список дней недели

day_to_number = {0: 'Mon', 1: 'Tue', 2: 'Wed', 3: 'Thu', 4: 'Fri', 5: 'Sat', 6: 'Sun'}
good_df['day'] = good_df['day'].map(day_to_number)

In [ ]:
# добавим категорию типа этажа

def floor_type(df):
    if df['floor'] == 1:
        return 'первый'
    elif df['floor'] == df['floors_total']:
        return 'последний'
    else:
        return 'другой'

good_df['floor_type'] = good_df.apply(floor_type, axis=1)

In [ ]:
# добавим расстояние до центра города

good_df['dist_to_center'] = np.round(good_df['city_centers_nearest'] / 1000)

In [ ]:
# проверям обновленный датафрейм

good_df.head().reset_index() 

**4 Проведем исследовательсктй анализ данных**

4.1 Воспользуемся методом описательной статистики, будем обращать внимание на средние значения и пределы нормальных значений.

In [ ]:
good_df.describe()

Построим гистограммы по следующим объектам:

In [ ]:
good_df.hist('last_price', bins=50, range=(0, 10000000), figsize=(10,5), grid=True)
good_df.hist('total_area', bins=10, range=(10, 120), figsize=(10,5), grid=True)
good_df.hist('rooms',  range=(0, 6), figsize=(10,5), grid=True)
good_df.hist('ceiling_height', bins=10, range=(2, 4), figsize=(10,5), grid=True)
good_df.hist('floors_total', bins=10, range=(0, 30), figsize=(10,5), grid=True)
good_df.hist('living_area',bins=20, range=(0, 150), figsize=(10,5), grid=True)
good_df.hist('floor', bins=10, range=(0, 30), figsize=(10,5), grid=True)
good_df.hist('kitchen_area', bins=20, range=(0, 40), figsize=(10,5), grid=True)
good_df.hist('balcony', range=(0, 5), figsize=(10,5), grid=True)
good_df.hist('airports_nearest', bins=20, range=(5000, 50000), figsize=(10,5), grid=True)
good_df.hist('dist_to_center', bins=10, range=(0, 20), figsize=(10,5), grid=True)
good_df.hist('parks_around_3000', bins=10, range=(0, 5), figsize=(10,5), grid=True)
good_df.hist('parks_nearest', bins=100, range=(0, 1), figsize=(10,5), grid=True)
good_df.hist('ponds_around_3000', bins=10, range=(0, 5), figsize=(10,5), grid=True)
good_df.hist('ponds_nearest', bins=100, range=(0, 1), figsize=(10,5), grid=True)

In [ ]:
good_df['floor_type'].value_counts().plot.bar()

In [ ]:
good_df['day'].value_counts().plot.bar()

In [ ]:
good_df['month'].value_counts().sort_index(ascending=True).plot.bar()

In [ ]:
good_df['year'].value_counts().sort_index(ascending=True).plot.bar()

Изучив гистограмм составим общие выводы об объектах недвижимости в Санкт-Петербурге и области:
* в среднем квартиры продавались по цене от 2,5 млн. до 5,5 млн. руб.
площать квартир от 40 до 60 метров, с объемом кухни 6-11 метров
* в продаже в основном 1 и 2 комнатные квартиры в 5-ти этажных домах
* высота потолков от 2,5 до 2,75 м
* до ближайшего аэропорта 12-25 км
* большая часть объявлений размещается на срок 30-60 дней
* не часто продаются квартиры на первых и последнем этажах
* объявления в основом размещаются в рабочие дни, а по времени года характерно повышение объявлений в первом квартале года, снижение летом, затем плавное повышение в третьем квартале и падение активности к новому году. Пиком продаж были 2017 и 2018 года, в 2019 было снижение активности более чем в два раза.
* в выборке много объявлений из области, где расстояние до центра небольшое, поэтому большинство значений до 2-х км
* территориальное расположение как правило не располагает наличие парка или пруда

4.2 Изучим подробнее столбец days_exposition, как быстро продавались квартиры.

In [ ]:
good_df.hist('days_exposition', bins=50, range=(1, 500), figsize=(10,5), grid=True);

In [ ]:
# уберем из расчетов нулевые значения

days_exposition = good_df.query('days_exposition > 0')['days_exposition']

In [ ]:
days_exposition.describe()

Средний срок продажи квартиры с даты публикации до снятия с продажи составляет 180 дней, медианное значение - 95 дней. Как видно из гистограммы большая часть квартир продается за 3 месяца. Пиковые значения в диапазоне 30, 90, 120 дней говорит о том, что объявления снимались с продажи по условловиям оплаченного времени размещения объявления. Объявления со сроком публикации свыше 180 дней можно считать слабо ликвидными.

4.3 Изучим какие факторы больше всего влияют на общую (полную) стоимость объекта.

In [ ]:
(df
    .plot(x='total_area',
    y='last_price',
    style='o',
    xlim=(10, 150),
    ylim=(0, 10000000),
    figsize=(12, 5),
    grid=True)
)
print('Корреляция с общей площадью: ', df['total_area'].corr(df['last_price']))

In [ ]:
(df
    .plot(x='living_area',
    y='last_price',
    style='o',
    xlim=(0, 150),
    ylim=(0, 10000000),
    figsize=(12, 5),
    grid=True)
)
print('Корреляция с жилой площадью: ', df['living_area'].corr(df['last_price']))

In [ ]:
(df
    .plot(x='rooms',
    y='last_price',
    style='o',
    xlim=(0, 10),
    figsize=(12, 5),
    grid=True)
)
print('Корреляция с количеством комнат: ', df['rooms'].corr(df['last_price']))

In [ ]:
(df
    .plot(x='kitchen_area',
    y='last_price',
    style='o',
    xlim=(0, 50),
    ylim=(0, 10000000),
    figsize=(12, 5),
    grid=True)
)
print('Корреляция с площадью кухни: ', df['kitchen_area'].corr(df['last_price']))

**Вывод: на стоимость объекта недвижимости сильнее всего влияет общая площадь.**

4.4 Посчитаем среднюю цену одного квадратного метра в 10 населённых пунктах с наибольшим числом объявлений. Выделим населённые пункты с самой высокой и низкой стоимостью квадратного метра.

In [ ]:
# top-10 населенных пунктов по количеству объявлений

good_df['locality_name'].value_counts().head(10)

In [ ]:
# напишем функцию которая будет определять среднюю стоимость квадратного метра в населенном пункте

def local_price_qm(city):
    local_price = good_df[good_df['locality_name'] == city]
    local_price_count = local_price['price_qm'].mean()
    return local_price_count

In [ ]:
info = {'city':['Санкт-Петербург', 'поселок Мурино', 'Кудрово', 'поселок Шушары', 'Всеволожск', 'Пушкин', 'Колпино',
                'поселок Парголово', 'Гатчина', 'Выборг'],
        'price_qm':[local_price_qm('Санкт-Петербург'), local_price_qm('поселок Мурино'),local_price_qm('Кудрово'),
                   local_price_qm('поселок Шушары'),local_price_qm('Всеволожск'),local_price_qm('Пушкин'),
                   local_price_qm('Колпино'),local_price_qm('поселок Парголово'),local_price_qm('Гатчина'),
                   local_price_qm('Выборг'),]}
                  
pd.DataFrame(info).sort_values(by='price_qm', ascending=False).reset_index(drop=True)

In [ ]:
# применил другой способ

good_df.groupby('locality_name')['price_qm'].agg(['count', 'mean']).sort_values(by='count', ascending=False).head(10)

Выведем десять населенных пунктов где стоимость квадратного метра самая высокая.

In [ ]:
good_df.groupby('locality_name')['price_qm'].agg('mean').sort_values(ascending=False)[:10]

Выведем десять населенных пунктов где стоимость квадратного метра самая низкая.

In [ ]:
good_df.groupby('locality_name')['price_qm'].agg('mean').sort_values(ascending=True)[:10]

4.5 Вычислим среднюю стоимость каждого километра для квартир в Санкт-Петербурге. Определим, как стоимость объектов зависит от расстояния до центра города.

In [ ]:
good_df_pivot = (good_df.query('locality_name == "Санкт-Петербург"')
            .pivot_table(index=['dist_to_center'], values='price_qm', aggfunc='mean')
            .plot(grid=True, figsize=(12, 5))
           )

In [ ]:
def dist_to_center(row):
    if 0 <= row['dist_to_center'] <= 8:
        return '8 км до центра'
    elif 8 <= row['dist_to_center'] <= 15:
        return '15 км до центра'
    else:
        return 'КАД и далее'

In [ ]:
good_df['dist_to_center_cat'] = good_df.apply(dist_to_center, axis=1)

In [ ]:
(
good_df.query('locality_name == "Санкт-Петербург"')
    .pivot_table(index=['dist_to_center_cat'], values='price_qm', aggfunc='mean')
    .sort_values(by='price_qm', ascending=False)
)

Вывод: как и следовало ожидать, чем ближе к центру находится объект недвижимости, тем выше цена.

In [ ]:
# дата первого и последнего обьявления

good_df['first_day_exposition'].min(), good_df['first_day_exposition'].max()

**5 Общий вывод**

**Резюме**
    
- средняя стоимость квартир, она составляет 2,5 - 5,5 млн. рублей, а диапазон предложений варьирует от 1,2 до 15 и выше млн
- в основном продаются 1 и 2 комнатные квартиры в 5-ти этажных домах, размер кухни в которых составляет от 6 до 11 метров с высотой потолков 2,65
- наибольшее влияние на стоимость квартиры оказывает общая площадь и расстояние до центра города
- больше всего объявлений о продаже было размещено в Санкт-Петербурге, поселке Мурино, Кудрово, Шушары
- самая высокая цена за кв м в Санкт-Петербурге, Сестрорецке, Зеленогорске и городе Пушкин
- самая низка цена за кв м в поселке Житково, Никольском и Боктитогорске 
- до ближайшего аэропорта в среднем 12-25 км  
- большая часть объявлений размещается на срок 30-90 дней
- не часто продаются квартиры на первых и последнем этажах
- территориальное расположение как правило не располагает наличие парка или пруда
- объявления в основом размещаются в рабочие дни, а по времени года характерно повышение объявлений в первом квартале года, снижение летом, затем плавное повышение в третьем квартале и падение активности к новому году. Пиком продаж были 2017 и 2018 года, в 2019 было снижение активности более чем в два раза

**Общий вывод**

Проведена предобработка архива данных о продаже квартир в Санкт-Петербурге и области за период с ноября 2014 по май 2019 года. Преждем чем приступить к исследованию, был проведен анализ всех данных датасета. Были обнаружены аномальные значения резко выделяющиеся на фоне других цифр, такие как высота потолков 100 м, цена за объект недвижимости более 75 млн., количество комнат превышаюшие пределы нормальных значений - 19 и общий размер объекта недвижимости - квартиры свыше 120 м кв. также не принимались к расчетам т.к. не входили в допустимый предел нормальных значений. Обработаны пропущенные значения для сохранения строк объявлений, совокупные потери данных составили 8% от первоначального датасета. В названиях населенных пунктах были устранены множество неявных дубликатов, присутствовали в названия "посёлки", "поселки" и т.д. В анализе данных применялись методы описательной статистики, использовались диаграммы размаха для определения размеров допустимых значений и аномальных выбросов данных. Графическими методами определяли корреляции, какие факторы больше всего влияют на общую цену обьекта недвижимости. Отсортировали объекты недвижимости с высокой и низкой стоимостью за 1 кв м по каждому населенному пункту и выделили по 10 лидеров в каждой категории.